# Open-Source Amalgamation and Single-Product Formats

## Amalgamation

The simplest form of export is "amalgamation" ... in which case the model and all necessary code to run are emitted as one big chunk.

In some cases, it's a single source code file that can be compiled on nearly any platform as a standalone program.
  * Classic amalgamation: MXNet + model code https://mxnet.incubator.apache.org/faq/smart_device.html#amalgamation-making-the-whole-system-a-single-file

In other cases, it's a chunk of consumable IR code that can be consumed in a common runtime:
  * H2O POJO export https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/productionizing.rst#pojo-quick-start
  * "Python Lambda Format" -- i.e., any single-function-call wrapper around a heap of Python + Python-compatible dependecies
    * *I'm not making this up* (not to say there's no role for this approach, but unlikely in the enterprise ML inference setting)
    * Examples
      * AWS Lambda, Google Cloud Functions, Azure Functions
      * https://fission.io/
      * http://fnproject.io/
      * https://github.com/kubeless/kubeless
      * http://openwhisk.incubator.apache.org/
      * etc.
    
And sometimes ... it's a coder implementing a model by hand and compiling it! (For simple, popular models, like linear/logistic regression, it's pretty easy once you have the model params.)

### What Does this Look Like? Let's Try It...

First, we need a model. So we'll train a quick linear regression on R's Diamonds dataset:

In [2]:
import pandas as pd

data = pd.read_csv('data/diamonds.csv')

X = data.carat
y = data.price

In [9]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X.values.reshape(-1,1), y)

This is a basic model for predicting price (in dollars) from weight (in carats).

The params are:

In [14]:
model.coef_, model.intercept_

(array([7756.42561797]), -2256.360580045403)

We can create a standalone ("amalgamated") version of this model using [SKompiler](https://pypi.org/project/SKompiler/)

In [23]:
from skompiler import skompile

expr = skompile(model.predict)
expr.to('sqlalchemy/postgresql')

'SELECT 7756.425617968435 * x1 + -2256.360580045403 AS y \nFROM data'

In [22]:
expr.to('sympy/c')

'y = 7756.4256179684353*x[0] - 2256.3605800454029;'

Here's a quick example with a decision tree. First, we'll create the model.

In [41]:
from sklearn.tree import DecisionTreeRegressor

model_tree = DecisionTreeRegressor(max_depth=4).fit(X.values.reshape(-1,1), y)

In [43]:
for line in skompile(model_tree.predict).to('sympy/c').split('\n'):
    print(line)

if (x[0] <= 0.99500000476837158) {
   if (x[0] <= 0.625) {
      if (x[0] <= 0.45499999821186066) {
         if (x[0] <= 0.375) {
            y = 703.12009314703926;
         }
         else {
            y = 961.50142450142448;
         }
      }
      else {
         if (x[0] <= 0.50499999523162842) {
            y = 1449.1314668289715;
         }
         else {
            y = 1738.4445772843724;
         }
      }
   }
   else {
      if (x[0] <= 0.86500000953674316) {
         if (x[0] <= 0.72500002384185791) {
            y = 2567.4392059553352;
         }
         else {
            y = 2945.0985116938341;
         }
      }
      else {
         if (x[0] <= 0.89499998092651367) {
            y = 3355.3200000000002;
         }
         else {
            y = 3954.4697792254797;
         }
      }
   }
}
else {
   if (x[0] <= 1.4950000047683716) {
      if (x[0] <= 1.1749999523162842) {
         if (x[0] <= 1.0349999666213989) {
            y = 5450.4928928159816;
         }
   

#### Pros and Cons: Amalgamation

Pros:
* Easy-to-understand concept
* Fairly portable
* Can be compact and performant
  * May be a good choice for extremely constrained embedded environments

Cons:
* Not interoperable with other high-level environments
* Violates separation of code (logic) from data (parameters)
* May not fit in well with enterprise manageability and operations needs

## Open-Source Single-Product Format(s)

I.e., a format which serves a specific product ecosystem, but is not intended to interoperate with other systems nor serve as a "standard"

*Examples:*

__SparkML + MLeap__
  * MLeap supports Spark, some scikit-learn models and some TensorFlow models
  * Represents models in a "MLeap Bundle"
  * MLeap runtime is a JAR that can run in any Java application (or by with a lightweight scoring wrapper provided by MLeap)

__TensorFlow + TensorFlow Serving__
  * TensorFlow models (created directly with TensorFlow or with Keras) serialize to a TF-specific protocol buffer representation
  * TensorFlow Serving loads the latest version of a model
    * TF Serving exposes a gRPC service and a REST endpoint
    
Within the intended ecosystem, we can easily explort and use a model, but we don't get portability across tools/ecosystems.